In [2]:
from unityagents import UnityEnvironment
import numpy as np
import pandas as pd
from dqn_agent import Agent
from collections import deque
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()
%matplotlib inline

import os
import torch

model_num = 8

os.system('mkdir -p model-{}'.format(model_num))

# env = UnityEnvironment(file_name="./Banana_Linux/Banana.x86_64")
# env = UnityEnvironment(file_name="./Banana_Linux_NoVis/Banana.x86_64")
env = UnityEnvironment(file_name='./Banana_Mac.app')


brain_name = env.brain_names[0]
brain = env.brains[brain_name]

state_size = 37
action_size = 4
seed = 42

BUFFER_SIZE = int(1e5)  # replay buffer size
BATCH_SIZE = 64         # minibatch size
GAMMA = 0.99            # discount factor
TAU = 1e-3              # for soft update of target parameters
LR = 5e-4               # learning rate 
UPDATE_EVERY = 4        # how often to update the network


agent = Agent(state_size, action_size, seed,
              BUFFER_SIZE, BATCH_SIZE, GAMMA, TAU, LR, UPDATE_EVERY)

def dqn(n_episodes=2000, max_t=2000, eps_start=1.0, eps_end=0.01, eps_decay=0.9):
    """Deep Q-Learning.
    
    Params
    ======
        n_episodes (int): maximum number of training episodes
        max_t (int): maximum number of timesteps per episode
        eps_start (float): starting value of epsilon, for epsilon-greedy action selection
        eps_end (float): minimum value of epsilon
        eps_decay (float): multiplicative factor (per episode) for decreasing epsilon
    """
    scores = []                        # list containing scores from each episode
    scores_window = deque(maxlen=100)  # last 100 scores
    eps = eps_start                    # initialize epsilon
    for i_episode in range(1, n_episodes+1):
    
        env_info = env.reset(train_mode=True)[brain_name]
        state = env_info.vector_observations[0]
        
        score = 0
        for t in range(max_t):
            action = agent.act(state, eps)
            env_info = env.step(action)[brain_name]
            
            next_state = env_info.vector_observations[0]
            reward = env_info.rewards[0]
            done = env_info.local_done[0]
            
            agent.step(state, action, reward, next_state, done)
            state = next_state
            score += reward
            if done:
                break 
                
        scores_window.append(score)       # save most recent score
        scores.append(score)              # save most recent score
        eps = max(eps_end, eps_decay*eps) # decrease epsilon
        print('\rEpisode {}\tAverage Score: {:.2f}'.format(i_episode, np.mean(scores_window)), end="")
        if i_episode % 100 == 0:
            print('\rEpisode {}\tAverage Score: {:.2f}'.format(i_episode, np.mean(scores_window)))
            torch.save(agent.qnetwork_local.state_dict(), 'model-{}/checkpoint_{:04d}.pth'.format(model_num, i_episode))
            
        if np.mean(scores_window)>=200:
            print('\nEnvironment solved in {:d} episodes!\tAverage Score: {:.2f}'.format(i_episode, np.mean(scores_window)))
            torch.save(agent.qnetwork_local.state_dict(), 'model-{}/checkpoint_{:04d}-score-{:.2f}.pth'.format(model_num, np.mean(scores_window) ,i_episode))
            break
    return scores

scores = dqn()

# plot the scores
fig = plt.figure()
ax = fig.add_subplot(111)
plt.plot(np.arange(len(scores)), scores)
plt.ylabel('Score')
plt.xlabel('Episode #')
plt.savefig('model-{}/scores.png'.format(model_num))
# plt.show()

df = pd.DataFrame({'episode':np.arange(len(scores)),'score':scores})
df.set_index('episode', inplace=True)
df.to_csv('model-{}/scores.csv'.format(model_num))

os.system('cp model.py model-{}/'.format(model_num))

env.close()

INFO:unityagents:
'Academy' started successfully!
Unity Academy name: Academy
        Number of Brains: 1
        Number of External Brains : 1
        Lesson number : 0
        Reset Parameters :
		
Unity brain name: BananaBrain
        Number of Visual Observations (per agent): 0
        Vector Observation space type: continuous
        Vector Observation space size (per agent): 37
        Number of stacked Vector Observation: 1
        Vector Action space type: discrete
        Vector Action space size (per agent): 4
        Vector Action descriptions: , , , 


Episode 10	Average Score: 0.20

KeyboardInterrupt: 